In [39]:
A = sc.parallelize(range(100), 4)
def f(x,sets):
    for i in sets:
        yield x + i
print(A.mapPartitionsWithIndex(f).collect())


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [34]:
print(A)

PythonRDD[32] at RDD at PythonRDD.scala:53


In [1]:
import psutil
# Data file at https://www.cse.ust.hk/msbd5003/data
lines = sc.textFile('../data/adj_noun_pairs.txt', 8)
#lines.count()
#lines.getNumPartitions()
#lines.take(5)

In [2]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()
#pairs.take(5)

PythonRDD[2] at RDD at PythonRDD.scala:53

In [7]:
N = pairs.count()

In [8]:
N

3162674

In [9]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2) \
                  .filter(lambda pf: pf[1] >= 100)

In [10]:
pair_freqs.take(5)

[(('16th', 'century'), 950),
 (('civil', 'war'), 2236),
 (('social', 'class'), 155),
 (('sixteenth', 'century'), 137),
 (('late', '1970'), 444)]

In [11]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

In [12]:
n_dict = n_freqs.collectAsMap()
a_dict = a_freqs.collectAsMap()
a_dict['violent']

1191

In [13]:
a_freqs.take(5)

[('social', 7413),
 ('free', 6635),
 ('stoic', 72),
 ('good', 7499),
 ('other', 75260)]

In [14]:
print(n_freqs.count())
print(a_freqs.count())
print(pair_freqs.count())

106333
104304
2012


In [15]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
n_dict = sc.broadcast(n_freqs.collectAsMap())   #python's map data structure.
a_dict = sc.broadcast(a_freqs.collectAsMap())
a_dict.value['violent']

1191

In [16]:
from math import *

# Computing the PMI for a pair.
def pmi_score(pair_freq):
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(a_dict.value[w1]*n_dict.value[w2]), 2) #divided by N in nominator.
    return pmi, (w1, w2)

In [17]:
# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)

In [18]:
# Printing the most strongly associated pairs. 
scored_pairs.top(10)

[(14.41018838546462, ('magna', 'carta')),
 (13.071365888694997, ('polish-lithuanian', 'Commonwealth')),
 (12.990597616733414, ('nitrous', 'oxide')),
 (12.64972604311254, ('latter-day', 'Saints')),
 (12.50658937509916, ('stainless', 'steel')),
 (12.482331020687814, ('pave', 'runway')),
 (12.19140721768055, ('corporal', 'punishment')),
 (12.183248694293388, ('capital', 'punishment')),
 (12.147015483562537, ('rush', 'yard')),
 (12.109945794428935, ('globular', 'cluster'))]